- 扫描目标路径
- 拿到所有图片路径
- 获得图片的分辨率信息
- 根据分辨率将其分类到对应的数组里
- 根据分组移动目标文件到新文件夹（根据分辨率创建）

获得需要的状态数据

In [1]:
import os
from StatusChecker.TraditionalStatusChecker import TraditionalStatusChecker
status_list = list(filter(lambda name: (name.startswith("ASC_STATUS_") and name != 'ASC_STATUS_UNKNOWN'),
                                  dir(TraditionalStatusChecker)))
status_list = list(map(lambda name: getattr(TraditionalStatusChecker, name), status_list))

print(status_list)
print(f"total {len(status_list)} status")

['annihilation_settlement', 'battle_settlement', 'fighting', 'level_selection', 'level_up', 'restore_sanity_medicine', 'restore_sanity_stone', 'team_up']
total 8 status


从 Log 文件夹导出数据

In [19]:
path_origin = os.path.join(os.getcwd(), "..", "log", "picture_log")
path_target = os.path.join(os.getcwd(), "..", "dataset", "unprocessed")
if not os.path.exists(path_target):
    os.mkdir(path_target)
file_list = os.listdir(path_origin)
print(f"total {len(file_list)} object need to move")
print("processing...")
for file in file_list:
    os.rename(os.path.join(path_origin, file), os.path.join(path_target, file))
print("done")

total 0 object need to move
processing...
done


并重命名数据
- 从可能存在的上下文恢复命名数据
- 根据命名数据更新图片名称
- 顺便访问一下分辨率数据
- 保存命名数据

In [4]:
import pickle
rename_dic = {}
rename_dic_serial_path = os.path.join(os.getcwd(), "..", "dataset", "rename_dic_serial")
if os.path.exists(rename_dic_serial_path):
    print("found recent rename data, read from serial")
    with open(rename_dic_serial_path, "rb") as rename_dic_serial_file:
        rename_dic = pickle.load(rename_dic_serial_file)
else:
    print("recent rename data not found, reinitialize one")
    for status in status_list:
        rename_dic[status] = 0
path_origin = os.path.join(os.getcwd(), "..", "dataset", "unprocessed")
path_target = os.path.join(os.getcwd(), "..", "dataset", "processed")
file_list = os.listdir(path_origin)
print(f"process name...")
file_status_dic = {}
for file in file_list:
    flag_found = False
    for status in status_list:
        if status in file:
            file_status_dic[file] = status # 识别出状态的就放入dict等待后续处理
            flag_found = True
            break
    if flag_found:
        flag_found = False
    else:
        print(f"error: not recognized name {file}, skipped")
print("done")
print(f"total {len(file_status_dic.keys())} objects need to rename")
print("processing...")

import cv2 as cv
# image_resolution_dic = {} # 分辨率数据需要的储存点

for file in file_status_dic.keys(): # 根据识别出状态的数量来处理

    image = cv.imread(os.path.join(path_origin, file))
    if image is None:
        print(f"invalid image {file}, skipped")
        continue
    if not os.path.exists(os.path.join(path_target, file_status_dic[file])):
        os.mkdir(os.path.join(path_target, file_status_dic[file]))
    rename_dic[file_status_dic[file]] += 1 # 命名规则增加1
    cv.imwrite(
        os.path.join(
            path_target,
            file_status_dic[file],
            f"{file_status_dic[file]}-{rename_dic[file_status_dic[file]]}.png"
        ),
        image
    )
    # # 读取其中的图片数据以便后续分类
    # resolution = f"{image.shape[1]}x{image.shape[0]}"
    # if resolution in image_resolution_dic.keys():
    #     image_resolution_dic[resolution].append(f"{file_status_dic[file]}-{rename_dic[file_status_dic[file]]}.png")
    # else:
    #     image_resolution_dic[resolution] = []
    #     image_resolution_dic[resolution].append(f"{file_status_dic[file]}-{rename_dic[file_status_dic[file]]}.png")
    os.remove(os.path.join(path_origin, file)) # 删除源文件夹的数据
# 保存rename数据
print("saving rename data")
with open(rename_dic_serial_path, "wb") as rename_data_serial_file:
    pickle.dump(rename_dic, rename_data_serial_file)
print("done")

recent rename data not found, reinitialize one
process name...
error: not recognized name log-2020-6-13-14-58-36-unknown.png, skipped
error: not recognized name log-2020-6-13-14-58-36-unrecognized status for ArkngithsStatusChecker.png, skipped
error: not recognized name log-2020-6-13-15-17-25-unknown.png, skipped
error: not recognized name log-2020-6-13-15-17-25-unrecognized status for ArkngithsStatusChecker.png, skipped
error: not recognized name log-2020-6-15-20-41-35-unknown.png, skipped
error: not recognized name log-2020-6-15-20-41-35-unrecognized status for ArkngithsStatusChecker.png, skipped
error: not recognized name log-2020-6-15-21-0-14-unknown.png, skipped
error: not recognized name log-2020-6-15-21-0-14-unrecognized status for ArkngithsStatusChecker.png, skipped
error: not recognized name log-2020-6-15-21-0-24-unknown.png, skipped
error: not recognized name log-2020-6-15-21-0-24-unrecognized status for ArkngithsStatusChecker.png, skipped
error: not recognized name log-2020-

根据图片分辨率拆分

In [21]:
# import cv2 as cv

# image_resolution_dic = {} # 分辨率数据需要的储存点

# for file in file_list:
#     image = cv.imread(os.path.join(path_origin, file))
#     resolution = f"{image.shape[1]}x{image.shape[0]}"
#     if resolution in image_resolution_dic.keys():
#         image_resolution_dic[resolution].append(file)
#     else:
#         image_resolution_dic[resolution] = []
#         image_resolution_dic[resolution].append(file)
# path_origin = os.path.join(os.getcwd(), "..", "dataset", "processed-name")
# path_target = os.path.join(os.getcwd(), "..", "dataset", "processed-resolution")
#
# print(image_resolution_dic)
#
# for key in image_resolution_dic.keys():
#     print(key)

{'1600x900': ['level_selection-886.png', 'team_up-616.png', 'fighting-1048.png', 'fighting-1049.png', 'fighting-1050.png', 'fighting-1051.png', 'fighting-1052.png', 'fighting-1053.png', 'fighting-1054.png', 'battle_settlement-624.png', 'level_selection-887.png', 'level_selection-888.png', 'fighting-1055.png', 'team_up-617.png', 'fighting-1056.png', 'fighting-1057.png', 'fighting-1058.png', 'fighting-1059.png', 'fighting-1060.png', 'battle_settlement-625.png', 'level_selection-889.png', 'level_selection-890.png', 'team_up-618.png', 'fighting-1061.png', 'fighting-1062.png', 'fighting-1063.png', 'fighting-1064.png', 'fighting-1065.png', 'fighting-1066.png', 'fighting-1067.png', 'fighting-1068.png', 'fighting-1069.png', 'level_selection-891.png', 'level_selection-892.png', 'team_up-619.png', 'battle_settlement-626.png', 'fighting-1070.png', 'fighting-1071.png', 'fighting-1072.png', 'fighting-1073.png', 'fighting-1074.png', 'fighting-1075.png', 'fighting-1076.png', 'battle_settlement-627.pn

按照新的分类来移动数据

In [22]:
# for key in image_resolution_dic.keys():
#     path_target_key = os.path.join(os.getcwd(), "..", "dataset", "processed", key)
#     print(f"process {key}")
#     if not os.path.exists(path_target_key):
#         os.mkdir(path_target_key)
#     for file in image_resolution_dic[key]:
#         os.rename(os.path.join(path_origin, file), os.path.join(path_target_key, file))
#     print(f"finished")

process 1600x900
finished


扫描 Dataset 目录下的所有图片，输出每个类型图片数据的数量

In [3]:
import os
path_target = os.path.join(os.getcwd(), "..", "dataset", "processed")
file_list = os.listdir(path_target)
info_dic = {}
for file in file_list:
    for status in status_list:
        if status in file:
            if status in info_dic.keys():
                info_dic[status] = info_dic[status] + 1
                break
            else:
                info_dic[status] = 1
                break
print(info_dic)
for key in info_dic.keys():
    print(f"{key} : {info_dic[key]}")



{'annihilation_settlement': 1, 'battle_settlement': 1, 'fighting': 1, 'level_selection': 1, 'level_up': 1, 'restore_sanity_medicine': 1, 'restore_sanity_stone': 1, 'team_up': 1}
annihilation_settlement : 1
battle_settlement : 1
fighting : 1
level_selection : 1
level_up : 1
restore_sanity_medicine : 1
restore_sanity_stone : 1
team_up : 1
